# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*.

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
display(df.head(10))
df.info()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Вывод: пропусков нет.

## Разбейте данные на выборки

Разделите исходные данные на обучающую, валидационную и тестовую выборки.

In [3]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

In [4]:
# отделил обучающаую выборку, взял 60%
features_train, features_rest, target_train, target_rest = train_test_split(features, target, test_size=0.40, random_state=12345)

# отделил валидационную и тестовую выборки 50/50
features_test, features_valid, target_test, target_valid = train_test_split(features_rest, target_rest, test_size=0.5, random_state=12345)


### Вывод

Разделил данные на три выборки: обучающую, валидационную и тестовую (60 %, 20%, 20%)

## Исследуйте модели

In [5]:
best_result = 0
best_depth = 0
for depth in range(1, 11):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train) #  обучил модель 

    predictions_valid = model.predict(features_valid) # предсказания на валидационной выборке 
    result = model.score(features_valid, target_valid) #  качество модели на валидационной выборке
    if result > best_result:
        best_result = result
        best_depth = depth
print("Наилучший Accuracy в модели 'Дерево решений'на валидационной выборке:", best_result)
print("depth:", best_depth)

Наилучший Accuracy в модели 'Дерево решений'на валидационной выборке: 0.7993779160186625
depth: 7


In [7]:
best_model = None
best_est = 0
for est in range(1, 11):
    model = RandomForestClassifier(random_state=12345, n_estimators=est) # модель с заданным количеством деревьев
    model.fit(features_train, target_train) # обучил модель на тренировочной выборке
    result = model.score(features_valid, target_valid) # качество модели на валидационной выборке
    if result > best_result:
        best_est = est# сохраните наилучшую модель
        best_result = result#  сохраните наилучшее значение метрики accuracy на валидационных данных
print("n_estimators =", est)
print("Наилучший Accuracy в модели 'Случайный лес'на валидационной выборке:", best_result)

n_estimators = 10
Наилучший Accuracy в модели 'Случайный лес'на валидационной выборке: 0.7993779160186625


In [27]:
model = LogisticRegression(random_state=12345) # модель логистической регрессии с параметром random_state=12345
model.fit(features_train, target_train) # обучите модель на тренировочной выборке
result = model.score(features_valid, target_valid) # получите метрику качества модели на валидационной выборке

print("Accuracy модели логистической регрессии на валидационной выборке:", result)

Accuracy модели логистической регрессии на валидационной выборке: 0.7402799377916018


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


### Вывод

Наилучший Accuracy показала модель 'Дерево решений' и 'Случайный лес' на валидационной выборке с показателем 0.799.

Почему то одинаковый показатель, если расчеты верно, просьба прокомментировать ревьюера возможно ли такое?

## Проверьте модель на тестовой выборке

Проверять буду модель 'Дерево решений'с показателем depth: 7, которые на валидационной выборке дали лучшие результаты.


In [28]:
model = DecisionTreeClassifier(random_state=12345, max_depth=7)
model.fit(features_train, target_train) #  обучил модель 
result = model.score(features_test, target_test) #  качество модели на тестовой выборке

print("Модель 'Дерево решений'на тестовой выборке показало качество:", result)

Модель 'Дерево решений'на тестовой выборке показало качество: 0.7822706065318819


На тестовой выборке показатель качества составил 0,782

## (бонус) Проверьте модели на адекватность

In [29]:
model = DummyClassifier(strategy="most_frequent")
model.fit(features_train, target_train)
model.predict(features_train)
model.score(features_train, target_train)

0.6924273858921162

### Общий вывод:

Построил несколько моделей для выявления лучший показателей. Наилучший Accuracy показала модель 'Дерево решений' с показателем depth = 7 на валидационной выборке с показателем 0,799. Проверил модель на тестовой выборке, показатель 0,78.